In [ ]:
# -*- coding: utf-8 -*-
#* <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
"""
* CS-370 | Current/ Emerging Trends in Computer Science - 2024
* Author: Ryan Hatch
* Date of Development: Mon Nov 14th 06:26:26 2024 
* Last Modified: Tues Nov 24th 12:21:21 2024
----------------------------------------------------------------------------------------------------
Description: This is a simple Convolutional Neural Network model that is used to classify images from the CIFAR 10 dataset.
The model is trained using the CIFAR 10 dataset and then evaluated on the test data. 
The model is saved to a JSON file and the weights are saved to an HDF5 file.
----------------------------------------------------------------------------------------------------
"""
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
# * NOTE: Use the below code to install TensorFlow 2.0 using pip in order to get started.
# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
"""-------------------------------------------------------------------------------------------------
                                #* Command to use:
pip install tensorflow==2.0.0
pip install --upgrade tensorflow
pip install matplotlib
pip install numpy
pip install keras
pip install h5py
pip install pillow
----------------------------------------------------------------------------------------------------
"""
#* <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>

# Import necessary libraries
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input
import matplotlib.pyplot as plt
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Define constants
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32
BATCH_SIZE = 128
NB_EPOCH = 50
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices (one-hot encoding)
Y_train = to_categorical(y_train, NB_CLASSES)
Y_test = to_categorical(y_test, NB_CLASSES)

# Normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Define the deeper convolutional neural network model
model = Sequential()

# First convolutional block
model.add(Conv2D(32, (3, 3), padding='same', activation='relu',
                 input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Second convolutional block
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Third convolutional block
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Fully connected layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES, activation='softmax'))

# Display model architecture
model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=OPTIM,
              metrics=['accuracy'])

# Define data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,          # Random rotations
    width_shift_range=0.1,      # Horizontal shifts
    height_shift_range=0.1,     # Vertical shifts
    horizontal_flip=True,       # Horizontal flips
    zoom_range=0.1              # Random zoom
)

# Compute quantities required for feature-wise normalization
datagen.fit(X_train)

# Train the model using data augmentation
history = model.fit(
    datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    epochs=NB_EPOCH,
    validation_data=(X_test, Y_test),
    verbose=VERBOSE
)

# Evaluate the model on test data
score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

# # Save model architecture to JSON
# model_json = model.to_json()
# with open('cifar10_architecture.json', 'w') as json_file:
#     json_file.write(model_json)
# print("Model architecture saved to cifar10_architecture.json")

# # Save model weights to HDF5
# model.save_weights('cifar10_weights.h5', overwrite=True)
# print("Model weights saved to cifar10_weights.h5")

# Save model architecture to JSON
model_json = model.to_json()
with open('cifar10_architecture.json', 'w') as json_file:
    json_file.write(model_json)
print("Model architecture saved to cifar10_architecture.json")

# Save model weights to HDF5 (corrected filename to end with `.weights.h5`)
model.save_weights('cifar10_weights.weights.h5', overwrite=True)
print("Model weights saved to cifar10_weights.weights.h5")


#* Version Two - Modified version with data augmentation and deeper network

# from keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.datasets import cifar10
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.optimizers import RMSprop
# import matplotlib.pyplot as plt

# # Constants
# IMG_CHANNELS = 3
# IMG_ROWS = 32
# IMG_COLS = 32
# BATCH_SIZE = 128
# NB_EPOCH = 20
# NB_CLASSES = 10
# VERBOSE = 1
# VALIDATION_SPLIT = 0.2
# OPTIM = RMSprop()

# # Load CIFAR-10 dataset
# (X_train, y_train), (X_test, y_test) = cifar10.load_data()
# print('X_train shape:', X_train.shape)
# print(X_train.shape[0], 'train samples')
# print(X_test.shape[0], 'test samples')

# # Preprocessing
# Y_train = to_categorical(y_train, NB_CLASSES)
# Y_test = to_categorical(y_test, NB_CLASSES)
# X_train = X_train.astype('float32') / 255
# X_test = X_test.astype('float32') / 255

# # Data augmentation
# datagen = ImageDataGenerator(
#     rotation_range=15,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=True)
# datagen.fit(X_train)

# # Deeper network
# model = Sequential()
# model.add(Conv2D(64, (3, 3), padding='same', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
# model.add(Activation('relu'))
# model.add(Conv2D(64, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(128, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(128, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(NB_CLASSES))
# model.add(Activation('softmax'))

# # Compile and train
# model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])
# model.fit(datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
#           epochs=NB_EPOCH, validation_split=VALIDATION_SPLIT, verbose=VERBOSE)

# # Evaluate
# score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
# print("Test score:", score[0])
# print("Test accuracy:", score[1])

#* Version One of the CNN model - Came with the book "Deep Learning with Python" by Francois Chollet

#  from keras.datasets import cifar10
#  from keras.utils import np_utils
#  from keras.models import Sequential
#  from keras.layers.core import Dense, Dropout, Activation, Flatten
#  from keras.layers.convolutional import Conv2D, MaxPooling2D
#  from keras.optimizers import SGD, Adam, RMSprop
#  import matplotlib.pyplot as plt

#  # CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
#  IMG_CHANNELS = 3

#  IMG_ROWS = 32
#  IMG_COLS = 32

#  #constant
#  BATCH_SIZE = 128
#  NB_EPOCH = 20
#  NB_CLASSES = 10
#  VERBOSE = 1
#  VALIDATION_SPLIT = 0.2
#  OPTIM = RMSprop()

#  #load dataset
#  (X_train, y_train), (X_test, y_test) = cifar10.load_data()
#  print('X_train shape:', X_train.shape)
#  print(X_train.shape[0], 'train samples')
#  print(X_test.shape[0], 'test samples')

# # convert to categorical
#  Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
#  Y_test = np_utils.to_categorical(y_test, NB_CLASSES)
#  # float and normalization
#  X_train = X_train.astype('float32')
#  X_test = X_test.astype('float32')
#  X_train /= 255
#  X_test /= 255

# # network
#  model = Sequential()
#  model.add(Conv2D(32, (3, 3), padding='same',
#  input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
#  model.add(Activation('relu'))
#  model.add(MaxPooling2D(pool_size=(2, 2)))
#  model.add(Dropout(0.25))

#  model.add(Flatten())
#  model.add(Dense(512))
#  model.add(Activation('relu'))
#  model.add(Dropout(0.5))
#  model.add(Dense(NB_CLASSES))
#  model.add(Activation('softmax'))
#  model.summary()

#  # train
#  model.compile(loss='categorical_crossentropy', optimizer=OPTIM,
#  metrics=['accuracy'])
#  model.fit(X_train, Y_train, batch_size=BATCH_SIZE,
#  epochs=NB_EPOCH, validation_split=VALIDATION_SPLIT,
#  verbose=VERBOSE)
#  score = model.evaluate(X_test, Y_test,
#  batch_size=BATCH_SIZE, verbose=VERBOSE)
#  print("Test score:", score[0])
#  print('Test accuracy:', score[1])

# #save model
#  model_json = model.to_json()
#  open('cifar10_architecture.json', 'w').write(model_json)
#  And the weights learned by our deep network on the training set
#  model.save_weights('cifar10_weights.h5', overwrite=True)